In [ ]:
import os

In [1]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction\\research'

In [2]:
os.chdir("../")

NameError: name 'os' is not defined

In [ ]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [ ]:
project_path = "C:/Users/RICH-FILES/Desktop/ml/Loan-Amount-Prediction"

os.chdir(project_path)

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth:  int
    min_samples_split: int
    min_samples_leaf: int
    target_column: float
    

In [ ]:
from credit_risk.constants import *
from credit_risk.utils.common import read_yaml, create_directories




In [ ]:
#creating a Configuration class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath   = CONFIG_FILE_PATH,
        params_filepath   = PARAMS_FILE_PATH,
        schema_filepath   = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)                       
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name = config.model_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,
            target_column=schema.name
        )
        
        return model_trainer_config
    
   
        
        
    
        

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import joblib
import os

class ModelTrainer:
    def __init__(self, config):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_data.dropna(inplace=True)
        test_data.dropna(inplace=True)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        numerical_features = ["Income", "Emp_length", "Rate", "Percent_income"]
        categorical_features = ["Home", "Intent"]

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", StandardScaler(), numerical_features),
                ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features)
            ]
        )

        # Check One-Hot Encoding Output Before Model Fitting
        train_x_transformed = preprocessor.fit_transform(train_x)
        test_x_transformed = preprocessor.transform(test_x)

        print("Transformed train_x shape:", train_x_transformed.shape)
        print("Transformed test_x shape:", test_x_transformed.shape)
        
        # Proceed with the pipeline
        model_pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("model", RandomForestRegressor(
                n_estimators=self.config.n_estimators,
                max_depth=self.config.max_depth,
                min_samples_leaf=self.config.min_samples_leaf,
                min_samples_split=self.config.min_samples_split,
                random_state=42))
        ])

        model_pipeline.fit(train_x, train_y)

        # Save the pipeline
        joblib.dump(model_pipeline, os.path.join(self.config.root_dir, self.config.model_name))

       
        
        #check target column
        #print(train_y[:5])
        
        #rfreg   = RandomForestRegressor(n_estimators=self.config.n_estimators, max_depth=self.config.max_depth, min_samples_leaf=self.config.min_samples_leaf, min_samples_split=self.config.min_samples_split ,random_state=42)
        
        #rfreg.fit(train_x, train_y)
       
        #joblib.dump(rfreg,os.path.join(self.config.root_dir, self.config.model_name))
        
        #Evaluation
        #'n_estimators': [100, 200, 300],
        #'max_depth': [10, 20, 30],
        #'min_samples_split': [2, 5, 10],
        #'min_samples_leaf': [1, 2, 4]
        #}
       
        #grid_search = GridSearchCV(rfreg, param_grid, cv=5, n_jobs=-1, verbose=2)
        #grid_search.fit(train_x, train_y)

        #print("Best Parameters:", grid_search.best_params_)

In [ ]:
#define pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e    

[2025-03-22 00:33:18,982: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-22 00:33:18,985: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-22 00:33:18,988: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-22 00:33:18,993: INFO: common: created directory at: artifacts]
[2025-03-22 00:33:18,995: INFO: common: created directory at: artifacts/model_trainer]
Transformed train_x shape: (20285, 14)
Transformed test_x shape: (5072, 14)
